In [1]:
# Load the TensorBoard notebook extension
#%load_ext tensorboard
#%reload_ext tensorboard
!kill $(ps aux | grep './ngrok' | awk '{print $2}')

In [2]:
import os
import tensorflow as tf
import tensorflow_addons as tfa
from ResNet50 import *
from LoadDataset import *
from tensorboard import notebook
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

#config = tf.compat.v1.ConfigProto()
#config.gpu_options.allow_growth = True

In [3]:
a = tf.Variable(2)
tf.print(a)
if tf.test.gpu_device_name(): 
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))

else:
   print("Please install GPU version of TF")

2
Default GPU Device:/device:GPU:0


In [4]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import numpy as np
import matplotlib.pyplot as plt
import datetime

from tensorflow.keras.layers import Dense, Flatten, Conv2D, Reshape
from tensorflow.keras import Model
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import tensorflow.keras.backend as K
from Squeeze_and_Excite import Squeeze_and_Excite 
from ResNet50 import *
from LoadDataset import *

#%load_ext tensorboard
#!rm -rf ./logs/

epoch = 100
batch_size = 32
learning_rate = 0.01
dataset = 10
momentum = 0.9
wd = 1e-2

#lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.1, decay_steps=100000, decay_rate=0.9)
#optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule, momentum=momentum)
#optimizer = tfa.optimizers.SGDW(learning_rate=lr_schedule, weight_decay=wd, momentum=momentum)
#optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, decay=decay_rate, momentum=momentum)
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer_sgd = tf.keras.optimizers.SGD(learning_rate=learning_rate)
optimizer_adam = tf.keras.optimizers.Adam(learning_rate=learning_rate)
optimizer = optimizer_adam

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')
test_top1 = tf.keras.metrics.SparseTopKCategoricalAccuracy(k=1, name='test_1_accuracy') 
test_top5 = tf.keras.metrics.SparseTopKCategoricalAccuracy(k=5, name='test_5_accuracy') 

@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
        K.set_learning_phase(1)
        predictions = model(images, training = True)
        loss =loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)
        
@tf.function
def test_step(images, labels):
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    K.set_learning_phase(0)
    predictions = model(images, training = False)
    t_loss =loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)
    test_top1(labels, predictions)
    test_top5(labels, predictions)


if __name__ == "__main__":

    save_model_dir = "./checkpoints_norm"

    print("loading data ...")
    train_X, train_lab, test_X, test_lab = get_data(dataset)
    print("normalizing data...")
    train_X, test_X = normalize(train_X, test_X)
    print("augmenting data...")
    train_X = data_augmentation(train_X)
    train_data = tf.data.Dataset.from_tensor_slices((train_X, train_lab)).batch(batch_size)
    test_data = tf.data.Dataset.from_tensor_slices((test_X, test_lab)).batch(batch_size)
    

    #model = ResNet50(include_top=True, weights=None, squeeze=False, squeeze_type='Normal')#, input_tensor = tf.data.Dataset.from_tensor_slices((train_X)))
    #input_tensor = tf.placeholder(tf.float32, shape = [None, train_X.shape[0], train_X.shape[1], train_X.shape[2]])
    model = ResNet50(include_top = True, squeeze = True, squeeze_type = 'normal', classes = dataset) #pre, identity, normal

    """
    features, label = iter(train_dataset).next()
    print("example features:", features[0])
    print("example label:", label[0])
    """

    checkpoint_dir = os.path.join(save_model_dir, "ckpt")
    checkpoint = tf.train.Checkpoint(optimizer=optimizer, model=model)
    manager = tf.train.CheckpointManager(checkpoint, checkpoint_dir, max_to_keep=3)
    #checkpoint.restore(manager.latest_checkpoint)
    #if manager.latest_checkpoint:
    #    print("Restaurado de {}".format(manager.latest_checkpoint))
    #else:
    #    print("Inicializando desde cero")

    current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
    test_log_dir = 'logs/gradient_tape/' + current_time + '/test'
    train_summary_writer = tf.summary.create_file_writer(train_log_dir)
    test_summary_writer = tf.summary.create_file_writer(test_log_dir)

loading data ...
Loading ./Datasets/cifar-10-batches-py/data_batch_1 : 10000.
Loading ./Datasets/cifar-10-batches-py/data_batch_2 : 10000.
Loading ./Datasets/cifar-10-batches-py/data_batch_3 : 10000.
Loading ./Datasets/cifar-10-batches-py/data_batch_4 : 10000.
Loading ./Datasets/cifar-10-batches-py/data_batch_5 : 10000.
Loading ./Datasets/cifar-10-batches-py/test_batch : 10000.
Train data: (50000, 32, 32, 3) (50000,)
Test data : (10000, 32, 32, 3) (10000,)
normalizing data...
augmenting data...


In [5]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

--2020-05-13 08:16:19--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.209.108.221, 52.54.124.219, 18.213.76.145, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.209.108.221|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.2’

ngrok-stable-linux- 100%[===================>]  13.13M  15.2MB/s    in 0.9s    

2020-05-13 08:16:20 (15.2 MB/s) - ‘ngrok-stable-linux-amd64.zip.2’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [6]:
logdir = "logs"
LOG_DIR = logdir
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

Traceback (most recent call last):
  File "<string>", line 1, in <module>
IndexError: list index out of range


In [7]:
    train_losses = []
    train_accs = []
    test_losses = []
    test_accs = []
    
    for i in range(epoch):
        #print(i)
        #if i == 1:
            #optimizer = optimizer_sgd
        for images, labels in train_data:
            train_step(images, labels)
        with train_summary_writer.as_default():
            tf.summary.scalar('loss', train_loss.result(), step=i+1)
            tf.summary.scalar('accuracy', train_accuracy.result(), step=i+1)

            
        for images, labels in test_data:
            test_step(images, labels)
        with test_summary_writer.as_default():
            tf.summary.scalar('loss', test_loss.result(), step=i+1)
            tf.summary.scalar('accuracy', test_accuracy.result(), step=i+1)
            tf.summary.scalar('top1error', (1 - test_top1.result())*100, step=i+1)
            tf.summary.scalar('top5error', (1 - test_top5.result())*100, step=i+1)
            

        template = 'Epoch {}, Train Loss: {}, Train Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
        print(template.format(i+1,
                        train_loss.result(),
                        train_accuracy.result()*100,
                        test_loss.result(),
                        test_accuracy.result()*100))

        template = 'Top1 Error: {}, Top5 Error: {}'
        print(template.format((1 - test_top1.result())*100,\
                            (1 - test_top5.result())*100))

        save_path = manager.save()

        train_losses.append(train_loss.result())
        train_accs.append(train_accuracy.result())
        test_losses.append(test_loss.result())
        test_accs.append(test_accuracy.result())

        # Reinicia las metricas para el siguiente epoch.
        train_loss.reset_states()
        train_accuracy.reset_states()
        test_loss.reset_states()
        test_accuracy.reset_states()


    t = np.linspace(1, epoch, num=epoch)
    plot1 = plt.figure(1)
    train_l = plt.plot(t, train_losses, 'b', label='training')
    test_l = plt.plot(t, test_losses, 'r', label='validation')
    plt.xlabel('epoch')
    plt.ylabel('Loss')
    plt.legend(handles=[train_l, test_l])
    plt.title('Loss evolution')
    plt.savefig('./Result_Pics/loss_train')
    plot2 = plt.figure(2)
    train_a = plt.plot(t, train_accs, 'b', label='training')
    test_a = plt.plot(t, test_accs, 'r', label='validation')
    plt.xlabel('epoch')
    plt.ylabel('Accuracy')
    plt.legend(handles=[train_l, test_l])
    plt.title('Accuracy evolution')
    plt.savefig('.//Result_Pics//acc_train')
    plt.show()
    
    a = 1

Epoch 1, Train Loss: 39233.15234375, Train Accuracy: 9.977999687194824, Test Loss: 2.3096718788146973, Test Accuracy: 10.0
Top1 Error: 90.0, Top5 Error: 50.0
Epoch 2, Train Loss: 2.305590867996216, Train Accuracy: 9.920000076293945, Test Loss: 2.305086374282837, Test Accuracy: 10.0
Top1 Error: 90.0, Top5 Error: 50.0
Epoch 3, Train Loss: 2.304503917694092, Train Accuracy: 9.878000259399414, Test Loss: 2.306164264678955, Test Accuracy: 10.0
Top1 Error: 90.0, Top5 Error: 50.0
Epoch 4, Train Loss: 2.304453134536743, Train Accuracy: 9.842000007629395, Test Loss: 2.3050594329833984, Test Accuracy: 10.0
Top1 Error: 90.0, Top5 Error: 50.0


KeyboardInterrupt: 